In [ ]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [ ]:
# Store the CSV you saved created in part one into a Data Frame

city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

In [ ]:
city_data_df.dtypes

In [ ]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [43]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]
# 2. Assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, 
              max_intensity=300, point_radius=4)

# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
# Heatmap of percent humidity

locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
# Heatmap of percent cloudiness

locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Heatmap of percent Wind Speed

locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

In [ ]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [ ]:
# Filter the dataset to find the cities that fit the criteria

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

In [ ]:
preferred_cities_df.count()

In [51]:
# create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates. 

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Puerto Suarez,BO,80.13,-18.9500,-57.8000,
13,San Policarpo,PH,79.41,12.1791,125.5072,
19,Road Town,VG,78.39,18.4167,-64.6167,
20,Kahului,US,81.97,20.8947,-156.4700,
24,Puerto Ayora,EC,76.89,-0.7393,-90.3518,
28,Georgetown,MY,85.95,5.4112,100.3354,
30,Mackay,AU,83.68,-21.1500,149.2000,
32,Cairns,AU,89.11,-16.9167,145.7667,
34,Ixtapa,MX,75.22,20.7000,-105.2000,
37,Kirakira,SB,85.41,-10.4544,161.9205,


In [53]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

In [54]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Puerto Suarez,BO,80.13,-18.9500,-57.8000,Hotel casa real
13,San Policarpo,PH,79.41,12.1791,125.5072,Agapita M. Nicart
19,Road Town,VG,78.39,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
20,Kahului,US,81.97,20.8947,-156.4700,Maui Seaside Hotel
24,Puerto Ayora,EC,76.89,-0.7393,-90.3518,Finch Bay Galapagos Hotel
28,Georgetown,MY,85.95,5.4112,100.3354,Cititel Penang
30,Mackay,AU,83.68,-21.1500,149.2000,International Lodge Motel
32,Cairns,AU,89.11,-16.9167,145.7667,Mantra Trilogy Cairns
34,Ixtapa,MX,75.22,20.7000,-105.2000,hotel la terminal
37,Kirakira,SB,85.41,-10.4544,161.9205,SanBiz Lodge


In [61]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [62]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))